In [182]:
import ctypes

def fp32_string(data):
    return bin(ctypes.c_uint32.from_buffer(ctypes.c_float(data)).value)

fp32_string(178.125)

'0b1000011001100100010000000000000'

### fp32二进制字符串转换为十进制数
* 32 bit string -> (-1) ** sign * (1 + fraction) ** (exponent - bias) -> binary value -> decimal value
* 为什么要有偏移量bias？阶码是无符号整数，但是阶数是可以为负数的，加上偏移量是为了把负的阶数用无符号数（阶码）表示。观山听白
阶码是移码，可以省去一位符号
* ref: https://blog.csdn.net/tercel_zhang/article/details/52537726
* ref: https://zhuanlan.zhihu.com/p/144697348位以表示负数

In [183]:
def bin_to_dec(b):
    p = b.index('.')
    return int(b[:p],2)+int(b[p+1:],2)/2**len(b[p+1:])

def str_to_bin(string, exponet_bit, fraction_bit):
    sign = int(string[0])
    fraction = int(string[-fraction_bit:],10) / 10**fraction_bit
    exponent = int(string[1:exponet_bit+1],2)
    bias = 127
    print(exponent)
    value = (-1) ** sign * (1 +  fraction) * 10 ** (exponent - bias)
    return value
    
exponent_bit = 8
fraction_bit = 23


string = "01000011001100100010000000000000"
value = str_to_bin(string, exponent_bit, fraction_bit)
bin_to_dec(str(value))

134


178.125

### fp32二进制字符串先截断为bf16二进制字符串，再转换为十进制数

In [186]:
def bin_to_dec(b):
    p = b.index('.')
    return int(b[:p],2)+int(b[p+1:],2)/2**len(b[p+1:])

def str_to_bin(string, exponet_bit, fraction_bit):
    sign = int(string[0])
    fraction = int(string[-fraction_bit:],10) / 10**fraction_bit
    exponent = int(string[1:exponet_bit+1],2)
    bias = 127
    value = (-1) ** sign * (1 +  fraction) * 10 ** (exponent - bias)
    return value

def half_bf16(string):
    return string[:16]

exponent_bit = 8
fraction_bit = 7


string = "01000011001100100010000000000000"
half_string = half_bf16(string)
value = str_to_bin(half_string, exponent_bit, fraction_bit)
bin_to_dec(str(value))

178.0

### fp32二进制字符串先截断为fp16二进制字符串，再转换为十进制数
* exponent_bit = 8 bias = 127 (fp32)
* exponent_bit = 7 bias = 63
* exponent_bit = 6 bias = 31
* exponent_bit = 5 bias = 15 (fp16)

In [187]:
def bin_to_dec(b):
    p = b.index('.')
    return int(b[:p],2)+int(b[p+1:],2)/2**len(b[p+1:])

def str_to_bin(string, exponet_bit, fraction_bit):
    sign = int(string[0])
    fraction = int(string[-fraction_bit:],10) / 10**fraction_bit
    exponent = int(string[1:exponet_bit+1],2)
    bias = 15
    value = (-1) ** sign * (1 +  fraction) * 10 ** (exponent - bias)
    return value

def half_fp16(string):
    return string[0] + bin(int(string[1:9], 2) - 112)[2:].zfill(5) + string[9:9+10]

exponent_bit = 5
fraction_bit = 10


string = "01000011001100100010000000000000"
half_string = half_fp16(string)
value = str_to_bin(half_string, exponent_bit, fraction_bit)
bin_to_dec(str(value))

178.125

## TODO
* 完成fp32 fp16 bf16的加减乘除
* 使用位运算完成fp32 fp16 bf16的相互转换（c++？）
* https://blog.csdn.net/q2519008/article/details/129264884
* https://blog.csdn.net/ysaeeiderup/article/details/124104042
* https://blog.csdn.net/weixin_45378258/article/details/118964215
* https://zhuanlan.zhihu.com/p/345631081